In [1]:
!pip install "gcsfs>=2021.7.0" git+https://github.com/pangeo-forge/pangeo-forge-recipes.git

  Cloning https://github.com/pangeo-forge/pangeo-forge-recipes.git to /tmp/pip-req-build-ku2l91ws
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Using cached fsspec-2021.7.0-py3-none-any.whl (118 kB)
  Using cached rechunker-0.4.2-py3-none-any.whl (19 kB)
  Using cached xarray-0.18.2-py3-none-any.whl (807 kB)
  Created wheel for pangeo-forge-recipes: filename=pangeo_forge_recipes-0.4.0-py3-none-any.whl size=35871 sha256=b1173e7529bfdbf910b891a785eff38ce24a0b6c7d91065be13753c98b1bd18b
  Stored in directory: /tmp/pip-ephem-wheel-cache-_kc63bzj/wheels/80/cd/b1/7d60228a14186fe40337bcce894f24f8dfd3a4eb0764300672
Successfully built pangeo-forge-recipes
  Attempting uninstall: xarray
    Found existing installation: xarray 0.16.2
    Uninstalling xarray-0.16.2:
      Successfully uninstalled xarray-0.16.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 0.8.5
    Uninstalling fsspec-0.8.5:
 

In [1]:
from recipe import recipe
from helpers import setup_logging, setup_targets

recipe = {"noaa_oisst/v2.1-avhrr": recipe}

setup_logging()

### `ValueError: I/O operation on closed file.`
This error, raised by `fsspec` but apparently caused by `h5py` closing a file, occurs when re-opening the same netCDF file more than once from a cache. It appears to be an issue **when reading from the GCS cloud cache, but __not__ when reading from a local filesystem cache**.

> I am not clear if this issue would arise when reading from all cloud caches, or only GCS, as I have only tested it with GCS thus far.

To compare this behavior against `h5py`'s interaction with a local filesystem:
1. In the next cell:
 - change `cache="cloud"` below to `cache="local"`
 - un-comment the two-line `for` loop
2. If the directory `zarr_build` exists, remove it with `!rm -r zarr_build`
3. Restart the kernel and run all cells.

In [2]:
rec = setup_targets(recipe, cache="cloud") 

# for input_name in rec.iter_inputs():
#    rec.cache_input(input_name)

Targets for recipe "noaa_oisst/v2.1-avhrr" set to:
    input_cache: gs://pangeo-forge-us-central1/pangeo-forge-cache/noaa_oisst/v2.1-avhrr
    target: zarr_build/noaa_oisst/v2.1-avhrr.zarr
    


In [3]:
rec.prepare_target()

pangeo_forge_recipes.recipes.xarray_zarr - INFO - Found an existing dataset in target
pangeo_forge_recipes.recipes.xarray_zarr - DEBUG - <xarray.Dataset>
Dimensions:  (lat: 720, lon: 1440, time: 2, zlev: 1)
Coordinates:
  * lat      (lat) float32 -89.88 -89.62 -89.38 -89.12 ... 89.38 89.62 89.88
  * lon      (lon) float32 0.125 0.375 0.625 0.875 ... 359.1 359.4 359.6 359.9
  * time     (time) datetime64[ns] 1981-09-01T12:00:00 1981-09-01T12:00:00
  * zlev     (zlev) float32 0.0
Data variables:
    anom     (time, zlev, lat, lon) float32 dask.array<chunksize=(2, 1, 720, 1440), meta=np.ndarray>
    err      (time, zlev, lat, lon) float32 dask.array<chunksize=(2, 1, 720, 1440), meta=np.ndarray>
    ice      (time, zlev, lat, lon) float32 dask.array<chunksize=(2, 1, 720, 1440), meta=np.ndarray>
    sst      (time, zlev, lat, lon) float32 dask.array<chunksize=(2, 1, 720, 1440), meta=np.ndarray>
Attributes: (12/37)
    Conventions:                CF-1.6, ACDD-1.3
    cdm_data_type:          

In [4]:
for chunk in rec.iter_chunks():
    rec.store_chunk(chunk)

pangeo_forge_recipes.recipes.xarray_zarr - INFO - Opening inputs for chunk (0,)
pangeo_forge_recipes.recipes.xarray_zarr - INFO - Opening input with Xarray (0,): 'https://www.ncei.noaa.gov/data/sea-surface-temperature-optimum-interpolation/v2.1/access/avhrr/198109/oisst-avhrr-v02r01.19810901.nc'
pangeo_forge_recipes.storage - INFO - Opening 'https://www.ncei.noaa.gov/data/sea-surface-temperature-optimum-interpolation/v2.1/access/avhrr/198109/oisst-avhrr-v02r01.19810901.nc' from cache
pangeo_forge_recipes.storage - DEBUG - file_opener entering first context for <contextlib._GeneratorContextManager object at 0x7ff7f4fa49d0>
pangeo_forge_recipes.storage - DEBUG - entering fs.open context manager for gs://pangeo-forge-us-central1/pangeo-forge-cache/noaa_oisst/v2.1-avhrr/fe866b608e5c7eafba93f06954124ba1-https_www.ncei.noaa.gov_data_sea-surface-temperature-optimum-interpolation_v2.1_access_avhrr_198109_oisst-avhrr-v02r01.19810901.nc
pangeo_forge_recipes.storage - DEBUG - FSSpecTarget.open yi

ValueError: I/O operation on closed file.